# NBA Data Wrangling

## Import relevant packages


In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import json
import pickle
import time
import nba_py
import re
from nba_py.player import PlayerList, PlayerGameLogs
from nba_py.game import Boxscore
from nba_py import team

## Clean Data
  
Cleaning the data scraped from NBA.com is going to be much simpler. The stats are split up nicely, the player names are already formatted the way we want them, its just a matter of unpacking the json and indexing by datetime. We will need to merge the gamelog and roster data after unpacking the raw pickle files as well as fill in some null values.

### Import NBA Packages and Data

In [2]:
#bring in list of players
players = PlayerList(league_id='00', only_current=0).info()

#from Data Acquisition
def getlogs(player, season):
        try:
            playerlogs = PlayerGameLogs(players.PERSON_ID.loc[players.DISPLAY_FIRST_LAST == player],season = season)
        except: 
            return ['ERROR']# Draft Kings Data Wrangling
        print(season, player)
        time.sleep(.5)
        return  playerlogs.info()

#read raw gamelogs data    
gamelogs = pickle.load(open('gamelogsraw.p','rb'))


### Clean Up Errors and Unpack Gamelogs

It doesn't make sense that the NBA API would give players that are in the season, and also not return values for them. First lets identify the players that returned errors.

In [3]:
#create a dictionary with season and player key value pairs
{(s,player) for s in gamelogs for player in gamelogs[s] if (type(gamelogs[s][player]) == list)}

{('2007-08', 'Channing Frye'),
 ('2007-08', 'Mehmet Okur'),
 ('2008-09', 'Alan Anderson'),
 ('2008-09', 'Jason Hart'),
 ('2008-09', 'Josh Smith'),
 ('2009-10', 'Jannero Pargo'),
 ('2010-11', 'Devin Ebanks'),
 ('2010-11', 'Kenyon Martin'),
 ('2010-11', 'Paul Pierce'),
 ('2011-12', 'Gerald Wallace'),
 ('2011-12', 'Tony Allen'),
 ('2011-12', 'Walker Russell'),
 ('2012-13', 'Matt Carroll'),
 ('2017-18', 'Ian Clark')}

A quick check on NBA.com/stats and you can see that these players did play games in those years. I will just run the scaping function again for these years and players to fill out the data.

In [4]:
#For every item in the dictionayry of dataframes that is a list (['ERROR']), pull the data for that player for the NBA API
#using the getlogs function
#This will run for about 15-20 seconds
gamelogs = {s:{player:gamelogs[s][player] if (type(gamelogs[s][player]) != list)  
               else getlogs(player, s) for player in gamelogs[s]} for s in gamelogs}
{s:player for s in gamelogs for player in gamelogs[s] if (type(gamelogs[s][player]) == list)}

2007-08 Channing Frye
2007-08 Mehmet Okur
2008-09 Alan Anderson
2008-09 Jason Hart
2008-09 Josh Smith
2009-10 Jannero Pargo
2010-11 Devin Ebanks
2010-11 Kenyon Martin
2010-11 Paul Pierce
2011-12 Gerald Wallace
2011-12 Tony Allen
2011-12 Walker Russell
2012-13 Matt Carroll
2017-18 Ian Clark


{}

### Unpack Gamelogs Data into 1 Table

First, let's take a look at one of the dataframes in the dictionary to see what steps to take in order to unpack it.

In [5]:
gamelogs['2015-16']['Stephen Curry'].head()

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,22015,201939,0021501227,"APR 13, 2016",GSW vs. MEM,W,30,15,24,0.625,...,3,4,6,2,0,2,2,46,19,1
1,22015,201939,0021501201,"APR 10, 2016",GSW @ SAS,W,35,13,22,0.591,...,5,5,5,2,0,4,4,37,7,1
2,22015,201939,0021501190,"APR 09, 2016",GSW @ MEM,W,34,7,22,0.318,...,9,9,8,1,0,2,1,17,10,1
3,22015,201939,0021501177,"APR 07, 2016",GSW vs. SAS,W,36,11,19,0.579,...,5,5,9,2,0,3,2,27,14,1
4,22015,201939,0021501163,"APR 05, 2016",GSW vs. MIN,L,43,7,25,0.280,...,4,6,15,3,0,3,4,21,3,1


- The goal here is to stack each of these DataFrames on top of each other in a datatime index with Player Name as its own column

In [6]:
#Every dataframe for each player will be combined into 1 table and assigned as a value to each season, keeping the 
#keeping the name of the player as the index 
gamelogsdf = {season:pd.concat(gamelogs[season]) for season in gamelogs}

#Now concatenate all the season tables into one big table, pd.concat wil keep the keys as the index and add a generic
#range index as well, drop that index to keep just the season and player names
gamelogsdf = (pd.concat(gamelogsdf)).reset_index(level=2,drop = True)
gamelogsdf.head().append(gamelogsdf.tail())

SEASON_ID Player_ID     Game_ID     GAME_DATE  \
2007-08 Aaron Brooks     22007    201166  0020701226  APR 16, 2008   
        Aaron Brooks     22007    201166  0020701208  APR 14, 2008   
        Aaron Brooks     22007    201166  0020701201  APR 13, 2008   
        Aaron Brooks     22007    201166  0020701184  APR 11, 2008   
        Aaron Brooks     22007    201166  0020701171  APR 09, 2008   
2017-18 Zhou Qi          22017   1627753  0021700140  NOV 05, 2017   
        Zhou Qi          22017   1627753  0021700123  NOV 03, 2017   
        Zhou Qi          22017   1627753  0021700111  NOV 01, 2017   
        Zhou Qi          22017   1627753  0021700080  OCT 28, 2017   
        Zhou Qi          22017   1627753  0021700032  OCT 21, 2017   

                          MATCHUP WL MIN FGM FGA  FG_PCT       ...       DREB  \
2007-08 Aaron Brooks  HOU vs. LAC  W  17   3   8   0.375       ...          1   
        Aaron Brooks    HOU @ UTA  L  14   2   5   0.400       ...          0   
        Aaron Brooks    HOU @ DEN  L  17   2   8   0.250       ...          0   
        Aaron Brooks  HOU vs. PHX  W  18   3   4   0.750       ...          2   
        Aaron Brooks  HOU vs. SEA  W  17   4   8   0.500       ...          4   
2017-18 Zhou Qi       HOU vs. UTA  W   5   1   2   0.500       ...          1   
        Zhou Qi         HOU @ ATL  W   7   0   3   0.000       ...          1   
        Zhou Qi         HOU @ NYK  W   5   1   3   0.333       ...          0   
        Zhou Qi         HOU @ MEM  L   3   0   1   0.000       ...          1   
        Zhou Qi       HOU vs. DAL  W   7   0   1   0.000       ...          3   

                     REB  AST STL BLK  TOV PF PTS PLUS_MINUS VIDEO_AVAILABLE  
2007-08 Aaron Brooks   1    1   2   0    0  2   8         -7               0  
        Aaron Brooks   0    3   0   0    1  4   8         -5               0  
        Aaron Brooks   0    3   0   0    1  3   7         -5               0  
        Aaron Brooks   2    3   0   0    1  4   7          7               0  
        Aaron Brooks   4    4   0   1    4  1  11         10               0  
2017-18 Zhou Qi        1    0   0   0    1  0   4          4               1  
        Zhou Qi        1    0   0   0    0  1   2        -15               1  
        Zhou Qi        0    0   0   0    1  2   3         -4               2  
        Zhou Qi        1    0   0   0    0  0   0          5               1  
        Zhou Qi        3    0   0   1    1  1   0         -7               1  

[10 rows x 27 columns]

In [7]:
#Shift the Multi-index into 2 separate columns by giving the indexes names, then resetting the index
gamelogsdf.index = gamelogsdf.index.set_names(['Season','Player'])
gamelogsdf = gamelogsdf.reset_index(inplace=False)
gamelogsdf.head()

,Season,Player,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,2007-08,Aaron Brooks,22007,201166,0020701226,"APR 16, 2008",HOU vs. LAC,W,17,3,...,1,1,1,2,0,0,2,8,-7,0
1,2007-08,Aaron Brooks,22007,201166,0020701208,"APR 14, 2008",HOU @ UTA,L,14,2,...,0,0,3,0,0,1,4,8,-5,0
2,2007-08,Aaron Brooks,22007,201166,0020701201,"APR 13, 2008",HOU @ DEN,L,17,2,...,0,0,3,0,0,1,3,7,-5,0
3,2007-08,Aaron Brooks,22007,201166,0020701184,"APR 11, 2008",HOU vs. PHX,W,18,3,...,2,2,3,0,0,1,4,7,7,0
4,2007-08,Aaron Brooks,22007,201166,0020701171,"APR 09, 2008",HOU vs. SEA,W,17,4,...,4,4,4,0,1,4,1,11,10,0


In [8]:
#Now that we have season and name columns, lets make the GAME_DATE column a data time index
#First convert the column to datetime format, only keeping the date
gamelogsdf.GAME_DATE = pd.to_datetime(gamelogsdf.GAME_DATE, format = "%b %d, %Y")

#Set the datetime column as the index and sort it
gamelogsdf = gamelogsdf.set_index('GAME_DATE')
gamelogsdf.index = pd.to_datetime(gamelogsdf.index)
gamelogsdf = gamelogsdf.sort_index()
gamelogsdf.head()

,Season,Player,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2007-10-30,2007-08,Paul Millsap,22007,200794,0020700003,UTA @ GSW,W,28,6,10,...,7,10,0,1,1,1,2,16,20,0
2007-10-30,2007-08,Channing Frye,22007,101112,0020700001,POR @ SAS,L,9,0,3,...,1,2,0,0,0,0,2,0,-11,0
2007-10-30,2007-08,Jason Hart,22007,2078,0020700003,UTA @ GSW,W,16,1,6,...,1,1,1,1,0,4,4,5,-3,0
2007-10-30,2007-08,Steve Blake,22007,2581,0020700001,POR @ SAS,L,25,4,7,...,6,6,4,0,0,3,4,9,-10,0
2007-10-30,2007-08,Joel Przybilla,22007,2038,0020700001,POR @ SAS,L,31,4,5,...,9,10,1,0,1,2,3,13,-3,0


### Make sure stat data are numeric type

In [9]:
#create tuples of column name and the unique types of the values in each column
[(col,set([type(x) for x in gamelogsdf[col]])) for col in list(gamelogsdf.columns)]

[('Season', {str}),
 ('Player', {str}),
 ('SEASON_ID', {str}),
 ('Player_ID', {int}),
 ('Game_ID', {str}),
 ('MATCHUP', {str}),
 ('WL', {str}),
 ('MIN', {int}),
 ('FGM', {int}),
 ('FGA', {int}),
 ('FG_PCT', {numpy.float64}),
 ('FG3M', {int}),
 ('FG3A', {int}),
 ('FG3_PCT', {numpy.float64}),
 ('FTM', {int}),
 ('FTA', {int}),
 ('FT_PCT', {numpy.float64}),
 ('OREB', {int}),
 ('DREB', {int}),
 ('REB', {int, float}),
 ('AST', {int}),
 ('STL', {int}),
 ('BLK', {int}),
 ('TOV', {int}),
 ('PF', {int}),
 ('PTS', {int}),
 ('PLUS_MINUS', {int}),
 ('VIDEO_AVAILABLE', {int})]

In [10]:
#convert all of the stat types fo pd.numeric in order to take averages and such
cols = ['MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
   'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
   'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']

for col in cols:
    gamelogsdf[col] = pd.to_numeric(gamelogsdf[col]) 
    
[(col,set([type(x) for x in gamelogsdf[col]])) for col in list(gamelogsdf.columns)]

[('Season', {str}),
 ('Player', {str}),
 ('SEASON_ID', {str}),
 ('Player_ID', {int}),
 ('Game_ID', {str}),
 ('MATCHUP', {str}),
 ('WL', {str}),
 ('MIN', {numpy.int64}),
 ('FGM', {numpy.int64}),
 ('FGA', {numpy.int64}),
 ('FG_PCT', {numpy.float64}),
 ('FG3M', {numpy.int64}),
 ('FG3A', {numpy.int64}),
 ('FG3_PCT', {numpy.float64}),
 ('FTM', {numpy.int64}),
 ('FTA', {numpy.int64}),
 ('FT_PCT', {numpy.float64}),
 ('OREB', {numpy.int64}),
 ('DREB', {numpy.int64}),
 ('REB', {numpy.float64}),
 ('AST', {numpy.int64}),
 ('STL', {numpy.int64}),
 ('BLK', {numpy.int64}),
 ('TOV', {numpy.int64}),
 ('PF', {numpy.int64}),
 ('PTS', {numpy.int64}),
 ('PLUS_MINUS', {numpy.int64}),
 ('VIDEO_AVAILABLE', {int})]

### Check for nulls


In [11]:
#print the amount of null values in each column
gamelogsdf.isnull().sum()

Season              0
Player              0
SEASON_ID           0
Player_ID           0
Game_ID             0
MATCHUP             0
WL                  0
MIN                 0
FGM                 0
FGA                 0
FG_PCT              0
FG3M                0
FG3A                0
FG3_PCT             0
FTM                 0
FTA                 0
FT_PCT              0
OREB                0
DREB                0
REB                19
AST                 0
STL                 0
BLK                 0
TOV                 0
PF                  0
PTS                 0
PLUS_MINUS          0
VIDEO_AVAILABLE     0
dtype: int64

### Fill in nulls
Looks like we only have 19 values in the entire dataset that are null values. This is pretty good news, but we should fill in those values. Instead of filling in the null values with 0, lets fill them in with the amount of rebounds each player was averaging that season.

In [12]:
gamelogsdf.loc[gamelogsdf.REB.isnull()].head()

,Season,Player,SEASON_ID,Player_ID,Game_ID,MATCHUP,WL,MIN,FGM,FGA,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2011-01-08,2010-11,JJ Redick,22010,200755,0021000542,ORL @ DAL,W,26,2,7,...,3,NaN,2,0,0,1,3,9,3,0
2011-01-08,2010-11,Gilbert Arenas,22010,2240,0021000542,ORL @ DAL,W,21,5,11,...,3,NaN,1,2,0,3,2,14,8,0
2011-01-08,2010-11,Ian Mahinmi,22010,101133,0021000542,DAL vs. ORL,L,2,2,2,...,0,NaN,0,0,0,0,0,5,7,0
2011-01-08,2010-11,Jason Kidd,22010,467,0021000542,DAL vs. ORL,L,32,2,8,...,5,NaN,7,3,0,2,0,6,-23,0
2011-01-08,2010-11,Hedo Turkoglu,22010,2045,0021000542,ORL @ DAL,W,39,5,8,...,3,NaN,17,5,1,2,3,13,11,0


In [13]:
#save dataframe of null rebounds
nullreb = gamelogsdf.loc[gamelogsdf.REB.isnull()]

#identify dates where rebounds were null
set([date for date in nullreb.index])

{Timestamp('2011-01-08 00:00:00')}

In [14]:
#group the data by season and player, fill in the null values with the rounded averae of the season
#You can't have half a rebound
gamelogsdf['REB'] = gamelogsdf.groupby(['Season','Player'])['REB'].transform(lambda x: x.fillna(round(x.mean())))

#since our index has duplicates for each date, we need to specify the sliced df 
#wherever it appears to avoid duplicate index error
fillrebdate = gamelogsdf['2011-01-08':'2011-01-08']

#show the rebound data for the players that had null values before
fillrebdate.loc[fillrebdate.Player.isin(list(nullreb.Player))][['Player','REB']]

,Player,REB
GAME_DATE,,
2011-01-08,JJ Redick,2.0
2011-01-08,Gilbert Arenas,3.0
2011-01-08,Ian Mahinmi,2.0
2011-01-08,Jason Kidd,4.0
2011-01-08,Hedo Turkoglu,4.0
2011-01-08,Jason Terry,2.0
2011-01-08,Ryan Anderson,6.0
2011-01-08,Brandon Bass,6.0
2011-01-08,Dominique Jones,1.0


In [15]:
gamelogsdf.isnull().sum()

Season             0
Player             0
SEASON_ID          0
Player_ID          0
Game_ID            0
MATCHUP            0
WL                 0
MIN                0
FGM                0
FGA                0
FG_PCT             0
FG3M               0
FG3A               0
FG3_PCT            0
FTM                0
FTA                0
FT_PCT             0
OREB               0
DREB               0
REB                0
AST                0
STL                0
BLK                0
TOV                0
PF                 0
PTS                0
PLUS_MINUS         0
VIDEO_AVAILABLE    0
dtype: int64

### Split Matchup Column

We should have a column showing the team of the player in that row and the opponent team as well. The MATCHUP column always has the team of the player first and the oppenent last.


In [16]:
#create a list for the players team by taking the first 3 letter of the string
plyrTeam = [m[:3] for m in gamelogsdf.MATCHUP]

#insert that list right before the MATCHUP column
gamelogsdf.insert(5,'team',plyrTeam)

#do the same with the opponenet team, taking the last 3 characters
plyrOpp = [m[-3:] for m in gamelogsdf.MATCHUP]
gamelogsdf.insert(6,'opp',plyrOpp)

#remove the MATCHUP column as it is now unneccessary
gamelogsdf = gamelogsdf.drop('MATCHUP',1)

gamelogsdf.head()

,Season,Player,SEASON_ID,Player_ID,Game_ID,team,opp,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2007-10-30,2007-08,Paul Millsap,22007,200794,0020700003,UTA,GSW,W,28,6,...,7,10.0,0,1,1,1,2,16,20,0
2007-10-30,2007-08,Channing Frye,22007,101112,0020700001,POR,SAS,L,9,0,...,1,2.0,0,0,0,0,2,0,-11,0
2007-10-30,2007-08,Jason Hart,22007,2078,0020700003,UTA,GSW,W,16,1,...,1,1.0,1,1,0,4,4,5,-3,0
2007-10-30,2007-08,Steve Blake,22007,2581,0020700001,POR,SAS,L,25,4,...,6,6.0,4,0,0,3,4,9,-10,0
2007-10-30,2007-08,Joel Przybilla,22007,2038,0020700001,POR,SAS,L,31,4,...,9,10.0,1,0,1,2,3,13,-3,0


### Convert WL (win-loss) to binary

In [17]:
#For a win the WL column will be 1, and a loss will be 0
gamelogsdf.WL = [1 if x == 'W' else 0 for x in gamelogsdf.WL]
gamelogsdf.head()

,Season,Player,SEASON_ID,Player_ID,Game_ID,team,opp,WL,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2007-10-30,2007-08,Paul Millsap,22007,200794,0020700003,UTA,GSW,1,28,6,...,7,10.0,0,1,1,1,2,16,20,0
2007-10-30,2007-08,Channing Frye,22007,101112,0020700001,POR,SAS,0,9,0,...,1,2.0,0,0,0,0,2,0,-11,0
2007-10-30,2007-08,Jason Hart,22007,2078,0020700003,UTA,GSW,1,16,1,...,1,1.0,1,1,0,4,4,5,-3,0
2007-10-30,2007-08,Steve Blake,22007,2581,0020700001,POR,SAS,0,25,4,...,6,6.0,4,0,0,3,4,9,-10,0
2007-10-30,2007-08,Joel Przybilla,22007,2038,0020700001,POR,SAS,0,31,4,...,9,10.0,1,0,1,2,3,13,-3,0


### Add year, month, and day columns

In [18]:
gamelogsdf.insert(0,'day',gamelogsdf.index.day)

gamelogsdf.insert(0,'month',gamelogsdf.index.month)

gamelogsdf.insert(0,'year',gamelogsdf.index.year)

gamelogsdf.head()

,year,month,day,Season,Player,SEASON_ID,Player_ID,Game_ID,team,opp,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2007-10-30,2007,10,30,2007-08,Paul Millsap,22007,200794,0020700003,UTA,GSW,...,7,10.0,0,1,1,1,2,16,20,0
2007-10-30,2007,10,30,2007-08,Channing Frye,22007,101112,0020700001,POR,SAS,...,1,2.0,0,0,0,0,2,0,-11,0
2007-10-30,2007,10,30,2007-08,Jason Hart,22007,2078,0020700003,UTA,GSW,...,1,1.0,1,1,0,4,4,5,-3,0
2007-10-30,2007,10,30,2007-08,Steve Blake,22007,2581,0020700001,POR,SAS,...,6,6.0,4,0,0,3,4,9,-10,0
2007-10-30,2007,10,30,2007-08,Joel Przybilla,22007,2038,0020700001,POR,SAS,...,9,10.0,1,0,1,2,3,13,-3,0


### Final cleanup + write to csv file

In [19]:
#change all column names to lower case
gamelogsdf.columns = [x.lower() for x in gamelogsdf.columns]

#frop the video_available column ,it is unnecessary for our purposes
gamelogsdf = gamelogsdf.drop('video_available',1)

In [20]:
#write to csv
gamelogsdf.to_csv('gamelogsclean.csv')

The gamelogs data is now all cleaned up, now we have to unpack the raw roster data and merge the player information into the gamelogs table.

## Merge gamelogs and roster data


In [21]:
#Import roster data
rosters = pickle.load(open('rostersraw.p','rb'))

### Unpack roster data
This will use the same techniques as the gamelogs unpacking

In [22]:
#concatenate all dataframes of each team in each season like we did for gamelogs
rostersdf = {season:pd.concat(rosters[season]) for season in rosters}

#do the same for every table in each season
rostersdf = (pd.concat(rostersdf)).reset_index(level=[1,2],drop = True)

#set the season keys as a column, the format of season should match that of gamelogs
rostersdf.index = rostersdf.index.set_names('season')
rostersdf = rostersdf.reset_index(inplace=False)

#drop the season column in the rosters data, it only shows the start year, different format than gamelogs
rostersdf = rostersdf.drop('SEASON', 1)

rostersdf.head()


,season,TeamID,LeagueID,PLAYER,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID
0,2007-08,1610612737,00,Josh Childress,1,G-F,6-8,210,"JUN 20, 1983",25.0,3,Stanford,2735
1,2007-08,1610612737,00,Joe Johnson,2,G,6-7,235,"JUN 29, 1981",27.0,6,Arkansas,2207
2,2007-08,1610612737,00,Acie Law,4,G,6-3,195,"JAN 25, 1985",23.0,R,Texas A&M,201151
3,2007-08,1610612737,00,Josh Smith,5,F,6-9,235,"DEC 05, 1985",22.0,3,Oak Hill Academy (VA),2746
4,2007-08,1610612737,00,Mario West,6,G,6-5,210,"JUN 19, 1984",24.0,R,Georgia Tech,201238


### Clean Columns

We're only concerned with the following columns: NUM, POSITION, HEIGHT, WEIGHT, AGE, EXP. We will eventually merge on the season and player columns. 

The TeamID column could cause problems if a player was traded midseason and has gamelogs for separate teams. we can merge the TeamID data using the TeamsList function if need be.

In [23]:
#extract only column we want
rostersdf = rostersdf[['season','PLAYER','NUM','POSITION','HEIGHT','WEIGHT','AGE','EXP']]
rostersdf.columns = [x.lower() for x in rostersdf.columns]
#check data types for each column
[(col,set([type(x) for x in rostersdf[col]])) for col in list(rostersdf.columns)]

[('season', {str}),
 ('player', {str}),
 ('num', {NoneType, str}),
 ('position', {str}),
 ('height', {str}),
 ('weight', {str}),
 ('age', {numpy.float64}),
 ('exp', {str})]

In [24]:
#convert num column to numeric
rostersdf.num = pd.to_numeric(rostersdf.num)

#convert height column to inches then to numeric
rostersdf.height = pd.to_numeric([12*int(x.split('-')[0]) + int(x.split('-')[1]) for x in rostersdf.height])

#convert weight to numeric
rostersdf.weight = pd.to_numeric(rostersdf.weight)

#exp column is years of experience, change the R value (rookie) to 0 and convert column to numeric
rostersdf.exp = pd.to_numeric([0 if x == 'R' else x for x in rostersdf.exp])

rostersdf.head()

,season,player,num,position,height,weight,age,exp
0,2007-08,Josh Childress,1.0,G-F,80,210,25.0,3
1,2007-08,Joe Johnson,2.0,G,79,235,27.0,6
2,2007-08,Acie Law,4.0,G,75,195,23.0,0
3,2007-08,Josh Smith,5.0,F,81,235,22.0,3
4,2007-08,Mario West,6.0,G,77,210,24.0,0


## Merge dataframes

Now that both the gamelogs and rosters tables are all cleaned up, we can merge them on the season a dn player columns.

In [25]:
#check if there are any duplicate players for a given season
rostersdf[rostersdf.duplicated(['season','player'])]

,season,player,num,position,height,weight,age,exp
205,2007-08,Marcus Williams,1.0,G,75,205,22.0,1
3945,2015-16,Spencer Dinwiddie,8.0,G,78,200,23.0,1
3953,2015-16,Cameron Bairstow,41.0,F-C,81,250,25.0,1


In [26]:
#drop the duplicates, this will cause extra rows during a merge
rostersdf = rostersdf.drop_duplicates(['season','player'])

In [27]:
#merge the two tables on season and player columns, reset index of gamelogs to not lose datetime
#keeping the date time index_col=True in merge has unwanted results
nbadf = pd.merge(gamelogsdf.reset_index(),rostersdf, how='left', on = ['season','player'])

#compare the shapes of the tables before and after, nbadf should have same number of rows as gamelogsdf, and 7 more columns
#we added one column from resetting the index and 6 more from the non merged on rostersdf columns
print(gamelogsdf.shape, rostersdf.shape, nbadf.shape)
nbadf.head()


(271173, 31) (4921, 8) (271173, 38)


,GAME_DATE,year,month,day,season,player,season_id,player_id,game_id,team,...,tov,pf,pts,plus_minus,num,position,height,weight,age,exp
0,2007-10-30,2007,10,30,2007-08,Paul Millsap,22007,200794,0020700003,UTA,...,1,2,16,20,24.0,F-C,80.0,258.0,23.0,1.0
1,2007-10-30,2007,10,30,2007-08,Channing Frye,22007,101112,0020700001,POR,...,0,2,0,-11,44.0,C,83.0,248.0,25.0,2.0
2,2007-10-30,2007,10,30,2007-08,Jason Hart,22007,2078,0020700003,UTA,...,4,4,5,-3,3.0,G,75.0,180.0,30.0,6.0
3,2007-10-30,2007,10,30,2007-08,Steve Blake,22007,2581,0020700001,POR,...,3,4,9,-10,2.0,G,75.0,172.0,28.0,4.0
4,2007-10-30,2007,10,30,2007-08,Joel Przybilla,22007,2038,0020700001,POR,...,2,3,13,-3,10.0,C,85.0,255.0,28.0,7.0


In [28]:
#Reset game date as a datetime index
nbadf = nbadf.set_index('GAME_DATE', drop=True)
nbadf.index = pd.to_datetime(nbadf.index)
nbadf = nbadf.sort_index()
nbadf.head()

,year,month,day,season,player,season_id,player_id,game_id,team,opp,...,tov,pf,pts,plus_minus,num,position,height,weight,age,exp
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2007-10-30,2007,10,30,2007-08,Paul Millsap,22007,200794,0020700003,UTA,GSW,...,1,2,16,20,24.0,F-C,80.0,258.0,23.0,1.0
2007-10-30,2007,10,30,2007-08,Channing Frye,22007,101112,0020700001,POR,SAS,...,0,2,0,-11,44.0,C,83.0,248.0,25.0,2.0
2007-10-30,2007,10,30,2007-08,Jason Hart,22007,2078,0020700003,UTA,GSW,...,4,4,5,-3,3.0,G,75.0,180.0,30.0,6.0
2007-10-30,2007,10,30,2007-08,Steve Blake,22007,2581,0020700001,POR,SAS,...,3,4,9,-10,2.0,G,75.0,172.0,28.0,4.0
2007-10-30,2007,10,30,2007-08,Joel Przybilla,22007,2038,0020700001,POR,SAS,...,2,3,13,-3,10.0,C,85.0,255.0,28.0,7.0


### Create Fantasy Score Column

Create this column based on DraftKings daily fantasy scoring system.

In [29]:
#create a funstions that returns the point values for a double double or triple double by aa player
def dblcount(pts,reb,ast,stl,blk):
    #counts the number of core stats that are greater than 10
    dbls = sum(x>=10 for x in [pts,ast,reb,stl,blk])
    
    #if double double, ad 1.5pts
    if dbls == 2:
        return 1.5
    #if triple double or more, 4.5pts
    elif dbls >= 3:
        return 4.5
    else:
        return 0

#create an array with all the fantasy scores based on only the counting stats
fscorebase = np.array(nbadf.pts + .5*nbadf.fg3m + 1.25*nbadf.reb + 1.5*nbadf.ast + 2*nbadf.stl + 2*nbadf.blk - .5*nbadf.tov)

#make an array with the points added by double or triple doubles
dblcnt = np.array([dblcount(p,a,r,s,b) for p,a,r,s,b in zip(nbadf.pts, nbadf.ast, nbadf.reb, nbadf.stl, nbadf.blk)])

#add base and dblcount arrays together to get final fantasy scores, assign to fscore column in dataframe
nbadf['fscore'] = fscorebase + dblcnt
nbadf.head()


,year,month,day,season,player,season_id,player_id,game_id,team,opp,...,pf,pts,plus_minus,num,position,height,weight,age,exp,fscore
GAME_DATE,,,,,,,,,,,,,,,,,,,,,
2007-10-30,2007,10,30,2007-08,Paul Millsap,22007,200794,0020700003,UTA,GSW,...,2,16,20,24.0,F-C,80.0,258.0,23.0,1.0,33.50
2007-10-30,2007,10,30,2007-08,Channing Frye,22007,101112,0020700001,POR,SAS,...,2,0,-11,44.0,C,83.0,248.0,25.0,2.0,2.50
2007-10-30,2007,10,30,2007-08,Jason Hart,22007,2078,0020700003,UTA,GSW,...,4,5,-3,3.0,G,75.0,180.0,30.0,6.0,7.75
2007-10-30,2007,10,30,2007-08,Steve Blake,22007,2581,0020700001,POR,SAS,...,4,9,-10,2.0,G,75.0,172.0,28.0,4.0,21.50
2007-10-30,2007,10,30,2007-08,Joel Przybilla,22007,2038,0020700001,POR,SAS,...,3,13,-3,10.0,C,85.0,255.0,28.0,7.0,29.50


In [30]:
#write clean dataframe to a csv file
nbadf.to_csv('nbadf.csv')

## Merge NBA and Draftkings Data
   
Lets make a separate dataset combining the DraftKings and NBA Data. Since the Draftkings data only goes back to the 2014-15 season, we'll only look at that subset of the NBA data. We want to merge on year, month, day and of course the player name. 

In [31]:
#import wrangled DraftKings data
dkdata = pd.read_csv('dkdataclean.csv').sort_values(['year','month','day','player'])

#subset nbadf from the start of the 2014 season
dknbadf = nbadf['2014-10':].sort_values(['year','month','day','player'])

In [32]:
#sorted list of all unique players in each dataset
dkplayers = sorted([x for x in dkdata.player.unique()])
nbaplayers = sorted([x for x in dknbadf.player.unique()])

#create a 2 column dataframe from the unique players
mismatch = pd.DataFrame([[p for p in dkplayers if p not in nbaplayers],[p for p in nbaplayers if p not in dkplayers]]).T
#name the columns
mismatch.columns = ['dk','nba']

#Print the number of NBA players that arent matched with dk players
print(len([x for x in mismatch.nba if x != None]))
mismatch

47


,dk,nba
0,A.J. Hammons,AJ Hammons
1,A.J. Price,AJ Price
2,Akil Mitchell,Amar'e Stoudemire
3,Amare Stoudemire,Andrew White III
4,Amida Brimah,CJ McCollum
5,Amile Jefferson,CJ Miles
6,Andrew White,CJ Wilcox
7,Andy Rautins,DeAndre' Bembry
8,Brandon Ashley,Derrick Jones Jr.
9,C.J. McCollum,Edy Tavares


In [33]:

#strips all punctuation from a name
def alpha(name):
    return ("".join(re.findall("[a-zA-Z]+", name)))

#list of nba players with only alphanumeric characters
alphanba = [alpha(x) for x in nbaplayers]

#given a name, return name in the nba player list if all alpha numeric characters match in order
def matchalpha(name):
    #uses stipped nba list, the list of unique names, an the alpha function
    global alphanba, nbaplayers, alpha
    
    #if stripped version of name is in nba list, return the formatting from the nba list
    if alpha(name) in alphanba:
        return nbaplayers[alphanba.index(alpha(name))]   
    
    #if no match just return the name as is
    return name

#strip suffixes from the nba names and put them into a list
nosuffixnba = [x.strip(' II') for x in nbaplayers]
nosuffixnba = [x.strip(' III') for x in nosuffixnba]
nosuffixnba = [x.strip(' Jr.') for x in nosuffixnba]

#function that returns NBA formatted player name if all but the suffix match
def matchnosuffix(name):
    #uses a list of nba names stripped of their suffixes
    global nosuffixnba
    nosuffix = name.strip(' II')
    nosuffix = nosuffix.strip(' III')
    nosuffix = nosuffix.strip(' Jr.')
    if nosuffix in nosuffixnba:
        return nbaplayers[nosuffixnba.index(nosuffix)]
    return name

In [34]:
#use match alpha function to replace some of the mismatching names
dkplayers = [matchalpha(name) for name in dkplayers]

#see differences in the player lists after replacing
mismatch = pd.DataFrame([[p for p in dkplayers if p not in nbaplayers],[p for p in nbaplayers if p not in dkplayers]]).T
mismatch.columns = ['dk','nba']
print('# unmatched NBA names', len([x for x in mismatch.nba if x != None]))

# unmatched NBA names 29


In [35]:
#match names that only vary in each dataset by the suffix
dkplayers = [matchnosuffix(name) for name in dkplayers]

mismatch = pd.DataFrame([[p for p in dkplayers if p not in nbaplayers],[p for p in nbaplayers if p not in dkplayers]]).T
mismatch.columns = ['dk','nba']
print('# unmatched NBA names', len([x for x in mismatch.nba if x != None]))


# unmatched NBA names 17


In [36]:
#hard code the rest of the unmatching names from the NBA dataset
new_names = {'Walter Tavares':'Edy Tavares','Luigi Datome':'Gigi Datome','Ishmael Smith':'Ish Smith','Jose Barea':'J.J. Barea',
             'Joseph Young':'Joe Young','Juan Hernangomez':'Juancho Hernangomez',
             'Louis Amundson':'Lou Amundson','Louis Williams':'Lou Williams','Maurice Williams':'Mo Williams',
             'Nazareth Mitrou-Long':'Naz Mitrou-Long','Nene Hilario':'Nene','Sheldon McClellan':'Sheldon Mac',
             'Timothe Luwawu':'Timothe Luwawu-Cabarrot','Vince Hunter':'Vincent Hunter','Wade Baldwin':'Wade Baldwin IV',
             'Wes Matthews':'Wesley Matthews','Guillermo Hernangomez':'Willy Hernangomez'}
dkplayers = [new_names[name] if name in new_names else name for name in dkplayers]

mismatch = pd.DataFrame([[p for p in dkplayers if p not in nbaplayers],[p for p in nbaplayers if p not in dkplayers]]).T
mismatch.columns = ['dk','nba']
print('# unmatched NBA names', len([x for x in mismatch.nba if x != None]))
mismatch

# unmatched NBA names 0


,dk,nba
0,Akil Mitchell,None
1,Amida Brimah,None
2,Amile Jefferson,None
3,Andy Rautins,None
4,Brandon Ashley,None
5,Carrick Felix,None
6,Chris Johnson,None
7,DeQuan Jones,None
8,Derek Willis,None
9,Devin Robinson,None


We can see that there are still 41 players that are in the DK data set, but not in the NBA. A quick look at all the names in the list can tell us that these are all 2 way contract guys that are also affiliated with the G-League. Since the NBA API doesn't have any data on them, we won't use them. They will disappear on a merge anyway.

In [37]:
dkdata.head()


,Unnamed: 0,year,month,day,position,player,dkpoints,salary,team,opp,score,min,stats,starter
40,2014-10-28,2014,10,28,PF,Aaron Gordon,17.75,4900,ORL,NOP,84-101,28,11pt 3rb 1st 1bl 3to 1trey 5-8fg 0-0ft,bench
50,2014-10-28,2014,10,28,SF,Al-Farouq Aminu,2.00,3000,DAL,SAS,100-101,5,0pt 2rb 1to 0-2fg 0-0ft,bench
51,2014-10-28,2014,10,28,PF,Alexis Ajinca,1.50,3000,NOP,ORL,101-84,2,2pt 1to 1-1fg 0-0ft,bench
28,2014-10-28,2014,10,28,PF,Anthony Davis,75.75,9500,NOP,ORL,101-84,36,26pt 17rb 2as 3st 9bl 10-22fg 6-9ft,starter
60,2014-10-28,2014,10,28,C,Aron Baynes,7.50,3000,SAS,DAL,101-100,18,4pt 2rb 1bl 2to 2-3fg 0-0ft,bench


In [38]:
#merge the nba data and dk data on as many matching columns as possible
dknbadf = pd.merge(dknbadf,dkdata,how='left', on = ['year','month','day','player','team','opp'])
print(dknbadf.shape)
dknbadf.head()

(100338, 46)


,year,month,day,season,player,season_id,player_id,game_id,team,opp,...,exp,fscore,Unnamed: 0,position_y,dkpoints,salary,score,min_y,stats,starter
0,2014,10,28,2014-15,Aaron Gordon,22014,203932,0021400001,ORL,NOP,...,0.0,17.75,2014-10-28,PF,17.75,4900.0,84-101,28,11pt 3rb 1st 1bl 3to 1trey 5-8fg 0-0ft,bench
1,2014,10,28,2014-15,Al-Farouq Aminu,22014,202329,0021400002,DAL,SAS,...,4.0,2.00,2014-10-28,SF,2.00,3000.0,100-101,5,0pt 2rb 1to 0-2fg 0-0ft,bench
2,2014,10,28,2014-15,Alexis Ajinca,22014,201582,0021400001,NOP,ORL,...,4.0,1.50,2014-10-28,PF,1.50,3000.0,101-84,2,2pt 1to 1-1fg 0-0ft,bench
3,2014,10,28,2014-15,Anthony Davis,22014,203076,0021400001,NOP,ORL,...,2.0,75.75,2014-10-28,PF,75.75,9500.0,101-84,36,26pt 17rb 2as 3st 9bl 10-22fg 6-9ft,starter
4,2014,10,28,2014-15,Aron Baynes,22014,203382,0021400002,SAS,DAL,...,2.0,7.50,2014-10-28,C,7.50,3000.0,101-100,18,4pt 2rb 1bl 2to 2-3fg 0-0ft,bench


In [39]:

dknbadf.index = dknbadf['Unnamed: 0']
dknbadf.index.name = 'date'
dknbadf.index = pd.to_datetime(dknbadf.index, format = '%Y-%m-%d').date
dknbadf = dknbadf.drop(['Unnamed: 0','stats','min_y'], 1)
dknbadf.head()


,year,month,day,season,player,season_id,player_id,game_id,team,opp,...,weight,age,exp,fscore,position_y,dkpoints,salary,score,min_y,starter
2014-10-28,2014,10,28,2014-15,Aaron Gordon,22014,203932,0021400001,ORL,NOP,...,220.0,19.0,0.0,17.75,PF,17.75,4900.0,84-101,28,bench
2014-10-28,2014,10,28,2014-15,Al-Farouq Aminu,22014,202329,0021400002,DAL,SAS,...,215.0,24.0,4.0,2.00,SF,2.00,3000.0,100-101,5,bench
2014-10-28,2014,10,28,2014-15,Alexis Ajinca,22014,201582,0021400001,NOP,ORL,...,248.0,27.0,4.0,1.50,PF,1.50,3000.0,101-84,2,bench
2014-10-28,2014,10,28,2014-15,Anthony Davis,22014,203076,0021400001,NOP,ORL,...,220.0,22.0,2.0,75.75,PF,75.75,9500.0,101-84,36,starter
2014-10-28,2014,10,28,2014-15,Aron Baynes,22014,203382,0021400002,SAS,DAL,...,260.0,28.0,2.0,7.50,C,7.50,3000.0,101-100,18,bench


In [46]:
dknbadf.to_csv('dknbaclean.csv')

## Visualizations

Our data is now cleaned up and ready for analysis. 

In [40]:
import matplotlib.pyplot as plt
import seaborn as sns